In [1]:
import pandas as pd

data = pd.read_csv('../data/Power_Plants.csv')
data.shape

(10847, 34)

In [2]:
data.head(5)

,X,Y,OBJECTID,Plant_Code,Plant_Name,Utility_Name,Utility_ID,sector_name,City,County,...,source_desc,tech_desc,Source,Period,Latitude,Longitude,Bio_MW,Geo_MW,Bat_MW,Other_MW
0,-160.497222,55.339722,10058,1,Sand Point,"TDX Sand Point Generating, LLC",63560,IPP Non-CHP,Sand Point,Aleutians East,...,"Petroleum = 1.4 MW, Wind = 0.4 MW",Petroleum Liquids; Onshore Wind Turbine;,"EIA-860, EIA-860M and EIA-923",202108,55.339722,-160.497222,NaN,NaN,NaN,NaN
1,-87.356823,33.458665,10059,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,...,Hydroelectric = 53 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202108,33.458665,-87.356823,NaN,NaN,NaN,NaN
2,-88.010300,31.006900,10060,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,...,"Coal = 1118.5 MW, Natural Gas = 1281.2 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",202108,31.006900,-88.010300,NaN,NaN,NaN,NaN
3,-86.283056,32.583889,10061,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,...,Hydroelectric = 224.1 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202108,32.583889,-86.283056,NaN,NaN,NaN,NaN
4,-85.970800,34.012800,10062,7,Gadsden,Alabama Power Co,195,Electric Utility,Gadsden,Etowah,...,Natural Gas = 130 MW,Natural Gas Steam Turbine,"EIA-860, EIA-860M and EIA-923",202108,34.012800,-85.970800,NaN,NaN,NaN,NaN


In [5]:
resources = ['hydroelectric', 'solar','wind']
data = data[data['PrimSource'].isin(resources)]


In [6]:
states = sorted(data['StateName'].unique())
state_groups = data.groupby(['StateName', 'PrimSource'])

In [8]:
state_dict = {}

for (state, res_type), group in state_groups:
    if state not in state_dict:
        state_dict[state] = {}
    state_dict[state][res_type] = group[['Latitude', 'Longitude']].values.tolist()

In [10]:
wind = data[data['PrimSource']=='wind'][['Latitude', 'Longitude']].values.tolist()
water = data[data['PrimSource']=='hydroelectric'][['Latitude', 'Longitude']].values.tolist()
solar = data[data['PrimSource']=='wind'][['Latitude', 'Longitude']].values.tolist()

[[38.033327, -102.537915],
 [41.835278, -106.242778],
 [45.121705, -95.053236],
 [42.8625, -72.9628],
 [42.4789, -71.9],
 [38.116415, -121.769031],
 [42.82579, -99.777467],
 [46.895, -96.7347],
 [44.6694, -87.6333],
 [36.120994, -84.338733],
 [46.0958, -119.0964],
 [40.9044, -96.6231],
 [43.034722, -94.140556],
 [43.848529, -99.24757],
 [37.0478, -121.1708],
 [35.060556, -118.389722],
 [35.075, -118.3158],
 [35.0697, -118.3167],
 [33.92194, -116.56222],
 [33.9131, -116.5631],
 [33.926111, -116.615],
 [35.0686, -118.3833],
 [33.9369, -116.6183],
 [33.9461, -116.6769],
 [33.914986, -116.71026],
 [33.881667, -116.460278],
 [33.934, -116.5781],
 [35.043889, -118.276388],
 [33.890913, -116.541515],
 [35.045, -118.2642],
 [35.0506, -118.2572],
 [35.1011, -118.3372],
 [35.1011, -118.3372],
 [35.0583, -118.3417],
 [35.075, -118.3417],
 [35.0583, -118.3083],
 [38.1167, -121.8333],
 [33.918395, -116.573906],
 [35.05, -118.2572],
 [33.942, -116.661],
 [35.0506, -118.1714],
 [35.0663, -118.339444]

In [12]:

state_groups = data.groupby(['StateName', 'PrimSource'])
state_dict = {}

for (state, res_type), group in state_groups:
    if state not in state_dict:
        state_dict[state] = {}
    state_dict[state][res_type] = group[['Latitude', 'Longitude']].values.tolist()
state_dict

{'Alabama': {'hydroelectric': [[33.458665, -87.356823],
   [32.583889, -86.283056],
   [33.7845, -86.0524],
   [33.2553, -87.4495],
   [32.6189, -86.2548],
   [33.425878, -86.337547],
   [32.9633, -86.5187],
   [32.680394, -85.911442],
   [32.806025, -86.444892],
   [33.9406, -87.1077],
   [32.535436, -85.887614],
   [34.172142, -85.753806],
   [32.5743, -85.8901],
   [32.100881, -87.399767],
   [34.421272, -86.393931],
   [34.8069, -87.3819],
   [31.4033, -86.479469],
   [31.361146, -86.518307],
   [32.661976, -85.091552],
   [32.60965, -85.0805],
   [32.324242, -86.784319],
   [33.258281, -85.616047],
   [34.79782, -87.62537]],
  'solar': [[34.831511, -87.842204],
   [32.876313, -85.38821],
   [31.331148, -85.73019],
   [33.626728, -85.969481],
   [34.610987, -86.611551],
   [34.94974, -86.868053]]},
 'Alaska': {'hydroelectric': [[58.3176, -134.101],
   [58.3269, -134.4631],
   [55.615208, -131.356111],
   [57.6861, -152.895],
   [61.475211, -149.15009],
   [58.1415, -133.737],
   [5